<a href="https://colab.research.google.com/github/RunatK/WineNet/blob/main/WineNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (13.0, 5.0)
import torch
import random
import numpy as np
import sklearn.datasets
from sklearn.model_selection import train_test_split

In [ ]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministics = True

In [ ]:
wine = sklearn.datasets.load_wine()
wine.data.shape
X_train, X_test, Y_train, Y_test = train_test_split(
    wine.data,
    wine.target,
    test_size=0.3, 
    shuffle=True
)

In [ ]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
Y_train = torch.LongTensor(Y_train)
Y_test = torch.LongTensor(Y_test)

In [ ]:
class WineNet(torch.nn.Module):
    def __init__(self, count_of_hidden_neurons):
        super(WineNet, self).__init__()
        self.fc1 = torch.nn.Linear(13, count_of_hidden_neurons)
        self.activ1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(count_of_hidden_neurons, count_of_hidden_neurons)
        self.activ2 = torch.nn.Sigmoid()
        self.fc3 = torch.nn.Linear(count_of_hidden_neurons, 3)
        self.sm = torch.nn.Softmax(dim=1)
    def forward(self, x):
        x = self.fc1(x)
        x = self.activ1(x)
        x = self.fc2(x)
        x = self.activ2(x)
        x = self.fc3(x)
        return x
    def inference(self, x):
        x = self.forward(x)
        x = self.sm(x)
        return x

In [ ]:
wine_net = WineNet(17)
loss = torch.nn.CrossEntropyLoss()
optim = torch.optim.Adam(wine_net.parameters(), lr=0.001)
batch_size = 5 

In [ ]:
for epoch in range(10000):
    order = np.random.permutation(len(X_train))
    for start_index in range(0, len(X_train), batch_size):
        optim.zero_grad()

        batch_indexes = order[start_index:start_index * batch_size]

        x_batch = X_train[batch_indexes]
        y_batch = Y_train[batch_indexes]

        preds = wine_net.forward(x_batch)

        loss_val = loss(preds, y_batch)
        loss_val.backward()

        optim.step()
    if(epoch%100 == 0):
        test_preds = wine_net.forward(X_test)
        test_preds = test_preds.argmax(dim=1)
        if ((test_preds == Y_test).float().mean() > 0.95):
            print((test_preds == Y_test).float().mean())
            break

tensor(0.9630)
